## Setup

In [ ]:
import getpass  # For input prompt not to show what is entered
import json     # Provide convinent functions to handle json objects 
import re       # For regular expression
import requests # To handle http requests

import warnings
warnings.simplefilter('ignore') # Because I am using unverified ssl certificates 

baseurl = 'https://localhost/api/v0.0.2' # Pick yours

## Exercise 1. Login

This is a POST request to the `/login` endpoint that requires 2 parameters: `username` and `password`  
The expected response is a json string that contains a token property.

In [ ]:
response = json.loads(requests.post(
    url = baseurl + '/login',
    json = {'username': input('username: '), 'password': getpass.getpass('password: ')},
    verify = False
).content.decode('ascii'))
print(response)

*Store the token in a variable for later*

In [ ]:
token = response['token']

## Exercise 2. Print all candidates candid

This is a GET request to /candidates

In [ ]:
response = json.loads(requests.get(
    url = baseurl + '/candidates',
    verify = False,
    headers = {'Authorization': 'Bearer %s' % token}
).content.decode('ascii'))

In [ ]:
print(json.dumps(response, indent=2, sort_keys=True))

In [ ]:
candidates = response['Candidates']
for candidate in candidates:
    print(candidate['CandID'])

## Exercise 3. Find all candidates and session with tsi instruments

This is a series of GET request

In [ ]:
instrument = 'test_all_fields'

for candidate in candidates:
    candid = candidate['CandID']
    response = json.loads(requests.get(
        url = baseurl + '/candidates/' + candid,
        verify = False,
        headers = {'Authorization': 'Bearer %s' % token}
    ).content.decode('ascii'))
    visit_labels = response['Visits']
    for visit_label in visit_labels:
        response = json.loads(requests.get(
            url = baseurl + '/candidates/' + candid + '/' + visit_label + '/instruments',
            verify = False,
            headers = {'Authorization': 'Bearer %s' % token}
        ).content.decode('ascii'))
        if instrument in response['Instruments']:
            response = json.loads(requests.get(
                url = baseurl + '/candidates/' + candid + '/' + visit_label + '/instruments/' + instrument,
                verify = False,
                headers = {'Authorization': 'Bearer %s' % token}
            ).content.decode('ascii'))
            print(json.dumps(response, indent=2, sort_keys=True))

## Exercise 4. Inputs instrument data for a candidate

In [ ]:
def getInstrumentData(candid, visit_label, instrument):
    return json.loads(requests.get(
        url = baseurl + '/candidates/' + candid + '/' + visit_label + '/instruments/' + instrument,
        verify = False,
        headers = {'Authorization': 'Bearer %s' % token}
    ).content.decode('ascii'))

instrument = 'test_all_fields'
candid = '300001'
visit_label = 'V01'

print(getInstrumentData(candid, visit_label, instrument))

#### PUT request containing all the fields

In [ ]:
# Get all the fields an meta data
json_input = getInstrumentData(candid, visit_label, instrument)

# Update one field
json_input[instrument]['numeric_1'] = int(json_input[instrument]['numeric_1']) + 1


r = requests.put(
    url = baseurl + '/candidates/' + candid + '/' + visit_label + '/instruments/' + instrument,
    json = json_input,
    verify = False,
    headers = {'Authorization': 'Bearer %s' % token}
)

print(r.status_code)
print(r.text)
print(getInstrumentData(candid, visit_label, instrument))

#### PATCH request containing some of the fields

In [ ]:
# Get all the meta data
json_input = getInstrumentData(candid, visit_label, instrument)['Meta']

# Add fields in the instrument object
json_input[instrument] = {}
json_input[instrument]['numeric_1'] = 45
json_input[instrument]['score_field_1'] = None

print(json.dumps(json_input, indent=2, sort_keys=True))


In [ ]:
r = requests.patch(
    url = baseurl + '/candidates/' + candid + '/' + visit_label + '/instruments/' + instrument,
    json = json_input,
    verify = False,
    headers = {'Authorization': 'Bearer %s' % token}
)

print(r.status_code)
print(r.text)
print(getInstrumentData(candid, visit_label, instrument))